In [3]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, concat, col, regexp_extract,  regexp_replace, coalesce, lit, date_format

In [4]:
spark = SparkSession.builder \
    .appName("Notebook Glue Job Simulation") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/11 22:45:43 WARN Utils: Your hostname, 01120230807W11I resolves to a loopback address: 127.0.1.1; using 172.24.33.137 instead (on interface eth0)
25/02/11 22:45:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/11 22:45:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
json_path = "../data/delay_prediction_example.json"
df = spark.read.option("multiline", "true").json(json_path)

In [7]:
df.printSchema()

root
 |-- flight_data: struct (nullable = true)
 |    |-- aircraft_code: string (nullable = true)
 |    |-- arrival: string (nullable = true)
 |    |-- arrival_date: string (nullable = true)
 |    |-- arrival_time: string (nullable = true)
 |    |-- carrier: string (nullable = true)
 |    |-- departure: string (nullable = true)
 |    |-- departure_date: string (nullable = true)
 |    |-- departure_time: string (nullable = true)
 |    |-- flight_duration: string (nullable = true)
 |    |-- flight_number: string (nullable = true)
 |-- prediction_data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- probability: string (nullable = true)
 |    |    |-- result: string (nullable = true)
 |    |    |-- subType: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [9]:
df_exploded = df.withColumn("prediction", explode(col("prediction_data")))


In [12]:
df_flattened = df_exploded.select(
    # Datos de vuelo
    col("flight_data.departure").alias("departure_airport"),
    col("flight_data.arrival").alias("arrival_airport"),
    col("flight_data.departure_date").alias("departure_date"),
    col("flight_data.departure_time").alias("departure_time"),
    col("flight_data.arrival_date").alias("arrival_date"),
    col("flight_data.arrival_time").alias("arrival_time"),
    col("flight_data.aircraft_code").alias("aircraft_code"),
    col("flight_data.carrier").alias("carrier"),
    col("flight_data.flight_number").alias("flight_number"),
    col("flight_data.flight_duration").alias("flight_duration"),
    # Datos de predicción
    col("prediction.id").alias("prediction_id"),
    col("prediction.probability").alias("delay_probability"),
    col("prediction.result").alias("delay_result"),
    col("prediction.subType").alias("sub_type"),
    col("prediction.type").alias("type"),
    # 📌 Agregar columnas de particionado
    col("flight_data.departure_date").substr(1, 4).alias("year"),
    col("flight_data.departure_date").substr(6, 2).alias("month"),
    col("flight_data.departure_date").substr(9, 2).alias("day"),
)

In [13]:
df_flattened.show()

+-----------------+---------------+--------------+--------------+------------+------------+-------------+-------+-------------+---------------+--------------------+-----------------+--------------------+------------+----------+----+-----+---+
|departure_airport|arrival_airport|departure_date|departure_time|arrival_date|arrival_time|aircraft_code|carrier|flight_number|flight_duration|       prediction_id|delay_probability|        delay_result|    sub_type|      type|year|month|day|
+-----------------+---------------+--------------+--------------+------------+------------+-------------+-------+-------------+---------------+--------------------+-----------------+--------------------+------------+----------+----+-----+---+
|              MAD|            RAK|    2024-09-22|      23:50:00|  2024-09-23|    00:55:00|          73H|     UX|         1423|         PT2H5M|TK1816NCEIST20200801|       0.26012117|LESS_THAN_30_MINUTES|flight-delay|prediction|2024|   09| 22|
|              MAD|         

In [14]:
spark.stop()